# The following is the data extraction and data cleaning from a scraped TripAdvisor database to perform a exploratory and quantitative market study.

In [1]:
#Packages to use:
import pandas as pd
import math
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

C:\Users\Juan Vallejo\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## For the data collection, we will use the scraper on the following link: https://github.com/juanm-vallejo/TripAdvisorScraper-

In [5]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
##Function calls Selenium to get the HTML source of the input link. Get the quantity of Restaurants in the city.
##We use selenium given that the information is in a flexbox, the Request library struggles to get the HTML in that item.

def Initial():
    """
    Calls the request Library, gets the link to scrapre as input. The output is headers, desired link and total of restaurants in the city.
    """ 
    header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
    }

    link = str(input("Insert the link of the desired restaurant's city"))
    driver = webdriver.Chrome()
    driver.get(link)
    rawSource = driver.page_source
    source = bs(rawSource, 'html.parser')
    rawTotalRestaurants = source.find_all('span',{"class": 'ffdhf b'})
    totalRestaurants = int(((str(rawTotalRestaurants)).split('"ffdhf b">')[1]).split('</')[0])
    return(header, link, totalRestaurants)
##Turns the link into an scrappeable one. The link need to be ordered in a quantity of 30 restaurantes per page and its muliples.

def arrangeLink(link):
    """
    Recieves the link, which was input on Initial function, output a scrapable Link.
    """ 
    emp_str = ""
    for char in link:
        if char.isdigit():
            emp_str = emp_str + char
    position = link.find(emp_str) + len(emp_str)
    newLink = link[:position] + '-oa%s' + link[position:]

    return(newLink)
## This functions scrapes all the pages which cointains restaurants in the city. Selenium would be slow to go into each link,
## therefore, here we use the Requests library to get all the HTML, we filter by the needed classes. And get a list with all
## the restaurants to scrape.

def AllHTML(header, link ,totalRestaurants):
    """
    Recieves headers to call request, link to be arranged, quantity of restaurants in the desired city to be scraped.
    Returns list of the needed classes with the restaurants.
    """ 

    listOfAllClass = []
    for pag in range(0, totalRestaurants, 30):
        pages = requests.get(arrangeLink(link) %(pag), headers= header)
        soups = bs(pages.text, 'html.parser')
        pageHTMLs = soups.find_all('div',{"class": 'emrzT Vt o'})
        listOfAllClass.append(pageHTMLs)
    classList = []
    for ClassGroup in listOfAllClass:
        for Class in ClassGroup:
            classList.append(Class)
             
    return(classList)
## First function, If approved, go to getLink
## This function tests if the restaurants pass the cusine speciallity filter. The typeFood variable can be 
## filled with the kind of cusines that Tripadvisor has. The string must be equal to the ones in Tripadvisor.  

typeFood = []

def getFoodType(pageHTML):
    """
    Recieves HTML, searchs for the cusine filters, returns True if the restaurant has the kind of cusine, passed if not.
    """ 
    type = pageHTML.find('div', {'class': 'bhDlF bPJHV eQXRG'})
    typeStr = str(type)
    start = typeStr.find('"ceUbJ"') + len('"ceUbJ"')
    end = typeStr.find('</span></span>')
    substring = typeStr[start:end]

    if not typeFood:
        return(True)
    else:
        for food in typeFood:
            if food.lower() in (substring.lower()).strip():
                return(True)
            else:
                pass
## This function searchs for all the links needed to enter to get the information of Tripadvisor.
## In the page where all restaurants are listed we dont have the required info. Therefore, we grt the 
## referenced link to open all the information.

def getLink(pageHTML):
    """
    Recieves HTML, searchs for the selected restaurant link, returns the link to scrape for the information on the selected restaurant.
    """ 
    link= pageHTML.find('a', {'class': 'bHGqj Cj b'})
    linkStr = (str(link))
    start = linkStr.find('href="') + len('href="')
    end = linkStr.find('" target')
    substring = linkStr[start:end]

    return("https://www.tripadvisor.com" + substring)
#All the following functions clean the date to have a clearer dataframe.

def listKitchen(var):
    """
    Input, Kitchen/cusine related info. Output, list of cusine offered.
    """

    if "class" in str(var):
        var1 = str(var).split('>')[1]
        var2 = var1.split('<')[0]
        var3 = list(var2.split(','))
        return (var3)
    else:
        return(list(var.split(',')))

def intReviews(var):
    """
    Input, reviews related info. Output, integer quantity of reviews.
    """

    if "class" in str(var):
        var1 = str(var).split('>')[1]
        var2 = var1.split(' review')[0]
        if ',' in var2:
            var4 = var2.replace(',', '')
            return(var4)
        else:
            var3 = int(var2)
        return(var3)
    else:
        return(int(var))

def floatOverall(var):
    """
    Input, reviews average. Output, float overall punctuation.
    """

    if "class" in str(var):
        var1 = str(var).split('>')[1]
        var2 = var1.split('<')[0]
        var3 = float(var2)
        return (var3)
    else:
        return(float(var))

def strPriceRange(var):
    """
    Input, price range information. Output, str with the range and currency.
    """

    if '€' in str(var):
        if "class" in str(var):
            var1 = str(var).split('>')[1]
            var2 = var1.split('<')[0]
            var3 = str(var2)
            return (var3)
        else:
            return(str(var))
    else:
        return('Na')

def listPrices(var):
    """
    Input, price list of the restaurant. Output, list with prices.
    """

    if 'class' in str(var) and '€' in str(var):
        newVar = []
        var1 = str(var)
        var2 = list(var1.split(','))
        for item in var2:
            var3 = item.split('</span>')
            for price in var3:
                if '€' in str(price):
                    var4 = price.split('€')[1]
                    newVar.append(float(var4))
        return(newVar)
    else:
        return('Na')

def listPunctuation(var):
    """
    Input, punctuation info. Output, ordered quantity of punctuations (Excelent, Very Good, Average, Poor, Terrible).
    """
    
    newVar = []
    var1 = str(var)
    var2 = var1.split(',')
    for item in var2:
        var3 = item.split('>')[1]
        var4 = var3.split('<')[0]
        newVar.append(var4)
    return(newVar)

## This function opens all the links, which were previously filtered, to get the restaurant's information.
## We use selenium to open the link to avoid Ajax errors. The function avoids also any index error for the changes in the HTML
## in some particular page.

def potCustomer(link):
    """
    Recieves Link of the restaurant information, calls selenium to open the link. Returns dictionary with the desired information. 
    """ 
    try:
        driver = webdriver.Chrome()
        driver.get(link)
        rawSource = driver.page_source
        rawSourceSoup = bs(rawSource, 'html.parser')
        rawName = driver.find_elements(By.CLASS_NAME, 'fHibz')
        rawInfo = driver.find_elements(By.CLASS_NAME, 'dyeJW')
        rawWebsite = driver.find_elements(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/div[3]/span[3]/span/a')
        name = rawName[0].text
        kitchen = (rawSourceSoup.find_all('div',{"class": 'cfvAV'})[1])
        position = rawInfo[1].text
        address = rawInfo[3].text
        phoneNumber = rawInfo[4].text
        website = rawWebsite[0].get_attribute('href')
        numReviews = (rawSourceSoup.find_all('div',{"class": 'cfxpI ui_column is-12-mobile is-4-desktop'})[0].find_all('a',{"class": 'dUfZJ'}))[0]
        reviewsGen = rawSourceSoup.find_all('span',{"class": 'fdsdx'})
        priceRange = (rawSourceSoup.find_all('div',{"class": 'cfvAV'})[0])
        prices = rawSourceSoup.find_all('span',{"class": 'dXMSb d'})
        puntuationRaw = rawSourceSoup.find_all('div',{"class": 'prw_rup prw_filters_detail_checkbox'})
        puntuation = puntuationRaw[0].find_all('span',{"class": 'row_num'})

        db = {'CompanyName': str(name), 'Cusine': listKitchen(kitchen), 'Importance': position, 'Address': address, 'PhoneNumber': phoneNumber, 'Website': website, 'NumReviews': intReviews(numReviews), 'Overall': floatOverall(reviewsGen[0]), 'PriceRange': strPriceRange(priceRange), 'Prices': listPrices(prices), 'Puntuation': listPunctuation(puntuation)}
        return(db)
    except IndexError:
        pass
    
## This function agroups all the above. It calls every other function to get the information into a database.

def dataframe(classList):
    """
    Recieves list of classes, calls getFoodType, getLink, potCustomer. Returns database in pandas format. 
    """ 
    
    df = pd.DataFrame()

    for restaurant in classList:
        if getFoodType(restaurant) is True:
            link = getLink(restaurant)
            db = potCustomer(link)
            df = df.append(db, ignore_index=True)
        else:
            pass
    return df

baseData = Initial()
classList = AllHTML(baseData[0], baseData[1] , baseData[2])
df = dataframe(classList)
df.to_csv('ScrapedData.cvs')

In [2]:
df = pd.read_csv(r'ScrapedData.cvs')

In [3]:
df.describe()

,Unnamed: 0,NumReviews,Overall
count,1924.000000,1924.000000,1924.000000
mean,961.500000,276.117464,4.146830
std,555.555278,618.235213,0.616243
min,0.000000,1.000000,1.000000
25%,480.750000,26.000000,4.000000
50%,961.500000,87.000000,4.000000
75%,1442.250000,258.000000,4.500000
max,1923.000000,10228.000000,5.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1924 entries, 0 to 1923
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1924 non-null   int64  
 1   CompanyName  1924 non-null   object 
 2   Cusine       1924 non-null   object 
 3   Importance   1924 non-null   object 
 4   Address      1924 non-null   object 
 5   PhoneNumber  1924 non-null   object 
 6   Website      1924 non-null   object 
 7   NumReviews   1924 non-null   float64
 8   Overall      1924 non-null   float64
 9   PriceRange   1924 non-null   object 
 10  Prices       1924 non-null   object 
 11  Puntuation   1924 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 180.5+ KB


## Data Cleaning

Continously we will do some arranges on the dataframe. First we will delete all the duplicated restaurants on 'Address'. There is no possible way two restaurants are in the exact same physical location. Then we will reset the index and do some sanitary cleaning. 

In [5]:
df = df.drop_duplicates(subset=['Address'])
df = df.drop(['Unnamed: 0'], axis=1)
df.reset_index(inplace=True)

In [6]:
## Function to arrange the ranking of the restaurant. It only leaves the position as an Int and no more text.

def ranking(x):
    test = True
    position = []
    string = ''
    while test is True:
        for char in str(x):
            if char == ',':
                pass
            else:
                if char.isdigit():
                    position.append(char)
                else:
                    test = False
                    break
        return(int(string.join([str(item) for item in position])))

## Fuction to split the CAP from the Address column. 
def cap(x):
    cha0 = x.split(',')
    for i in cha0:
        i = i.strip()
        if (len(i) == 5) and i.isdigit():
            return(int(i))  

## Function to have the prices on a list. 
def prices_Cusine(variable):
    try:
        x1 = variable.replace('[', '')
        x2 = x1.replace(']', '')
        x3 = x2.replace(" ", "")
        x4 = x3.split(',')

        return(x4)
    except IndexError:
        return('None')

## Function to have the price range on a list. 
def min_max(prices):
    x1 = prices.replace('€', '')
    x2 = x1.replace(' ','')
    x3 = x2.split('-')
    return(x3)

In [7]:
df.Importance = df.Importance.apply(lambda x: ranking(x.split('#')[1]))

df['cap'] = df.Address.apply(lambda x: cap(x))

df = df[((80121 <= df.cap) & (df.cap <= 80147))]

df.Prices = df.Prices.apply(lambda x: prices_Cusine(x))

df.Cusine= df.Cusine.apply(lambda x: prices_Cusine(x))

df.PriceRange = df.PriceRange.apply(lambda x: min_max(x))

Just to keep the order, we will reset again the index

In [8]:
df.reset_index(inplace=True)

In [9]:
## Now we have to arrange the Punctuation column. Which in Tripadvisor splits as following:

def Punctuation(punctList):
    try:
        x1 = punctList.replace('[', '')
        x2 = x1.replace(']', '')
        x3 = x2.replace("'","")
        x4 = x3.replace(" ", "")
        x5 = x4.split(',')
        return(x5)
    except: 
        print('Check')

ex = []
vg = []
avg = []
po = []
ter= []

for i in (df.Puntuation).apply(lambda x: Punctuation(x)):
    ex.append(i[0])
    vg.append(i[1])
    avg.append(i[2])
    po.append(i[3])
    ter.append(i[4])


trip = {'Excellent': ex, 'VeryGood': vg, 'Average': avg, 'Poor': po, 'Terrible': ter}

df0 = df
df1 = pd.DataFrame(trip)


In [10]:
df = pd.concat([df0, df1], axis=1)
df = df.dropna(subset=['CompanyName', 'Address'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      1492 non-null   int64  
 1   index        1492 non-null   int64  
 2   CompanyName  1492 non-null   object 
 3   Cusine       1492 non-null   object 
 4   Importance   1492 non-null   int64  
 5   Address      1492 non-null   object 
 6   PhoneNumber  1492 non-null   object 
 7   Website      1492 non-null   object 
 8   NumReviews   1492 non-null   float64
 9   Overall      1492 non-null   float64
 10  PriceRange   1492 non-null   object 
 11  Prices       1492 non-null   object 
 12  Puntuation   1492 non-null   object 
 13  cap          1492 non-null   float64
 14  Excellent    1492 non-null   object 
 15  VeryGood     1492 non-null   object 
 16  Average      1492 non-null   object 
 17  Poor         1492 non-null   object 
 18  Terrible     1492 non-null   object 
dtypes: flo

To make the analisis more interesting in the exploratory secction. We need to add some data from other sources. Continously, we have the hood and administrative zone of each restaurant according to where the street is located. This information is provided by https://www.comune.napoli.it/home, and it allow us to understand better the consumer according to the statistical data we can get from https://www.istat.it/. 

In [12]:
dfCopy = df
dfCopy.Address = dfCopy.Address.apply(lambda x: (x.split(',')[0]).split(' ',1))
tipoStrada = [i[0] for i in dfCopy.Address]
viaQuartiere = pd.read_excel(r'Stradario_del_Comune_di_Napoli_ordinato_per_municipalita_quartieri_toponimi_aggiornato_al_2_12_2019.xlsx')

In [13]:
## Function to return the street names on lower case: 

def toponimo(topo):
    try: 
        top = topo[1].lower()
        return(top)
    except IndexError:
        print(i)

## These two functions work together. The objective here is to get just the name of the street. 
def ki(x):
    try:
        lenght = len(x) - 1
        c = 0
        for i in x:
            if i.isdigit():
                t = x.replace(i, '!')
                c = c + 1
                return(t)
            if c == lenght:
                return(x)
    except:
        return(x)

def indirizzo(x):
    try:
        y = x.split('!')[0]
        return(y)
    except:
        return(x)


In [14]:
viaQuartiere = viaQuartiere.drop(columns='INTER')
viaQuartiere.TOPONIMO = viaQuartiere.TOPONIMO.apply(lambda x: x.lower())
dfCopy.Address = dfCopy.Address.apply(lambda x: toponimo(x))
dfCopy.Address = dfCopy.Address.apply(lambda x: indirizzo(ki(x)))
dfCopy['Address1'] = dfCopy.Address

['4', '8', '8', '9', '22']


Now we have just the street name on a colum in our dataframe. Since the names are not 100% equal to the once provided by the official italia page, we can use the FuzzyWuzzy library to search for the most similar words in the list. We will use a 70% similarity as approval for the word.  

In [15]:
## This function uses the FuzzyWuzzy library to find the most similar word. 
def get_ratio(x):
    name = viaQuartiere.TOPONIMO.tolist()

    try:
        possibilities = process.extract(x, name, limit=100, scorer=fuzz.token_set_ratio)
        for i in possibilities:
            if i[1] >= 70:
                return(i[0])
            else:
                return(i)
    except: 
        return((x, 'check'))

dfCopy.Address1 = dfCopy.Address1.apply(lambda x: get_ratio(x))

In [16]:
dfCopy1 = dfCopy.drop(columns='Address')

In [17]:
dfCopy1 = dfCopy1.rename(columns={"Address1":"Address"})
viaQuartiere = viaQuartiere.rename(columns={"TOPONIMO":"Address"})

Now with our dataframe cleaned and with a common column, we can proceed to merge them on this common column.

In [18]:
dfCopy2 = pd.merge(dfCopy1, viaQuartiere, on='Address', how='left', left_index=False, right_index=False)

In [19]:
dfCopy2

,level_0,index,CompanyName,Cusine,Importance,PhoneNumber,Website,NumReviews,Overall,PriceRange,...,cap,Excellent,VeryGood,Average,Poor,Terrible,Address,TIPOLOGIA,QUARTIERE,MUN
0,0,0,Il tempo del vino e delle rose,"['Italian', 'Cafe', 'International', 'Mediterr...",333,+39 380 713 4511,http://www.facebook.com/iltempodelvinoedellerose/,256.0,4.5,"[20, 40]",...,80135.0,26,9,1,2,3,dante,PIAZZA,Avvocata,2.0
1,0,0,Il tempo del vino e delle rose,"['Italian', 'Cafe', 'International', 'Mediterr...",333,+39 380 713 4511,http://www.facebook.com/iltempodelvinoedellerose/,256.0,4.5,"[20, 40]",...,80135.0,26,9,1,2,3,dante,PIAZZA,S.Giuseppe,2.0
2,0,0,Il tempo del vino e delle rose,"['Italian', 'Cafe', 'International', 'Mediterr...",333,+39 380 713 4511,http://www.facebook.com/iltempodelvinoedellerose/,256.0,4.5,"[20, 40]",...,80135.0,26,9,1,2,3,dante,PIAZZA,S.Lorenzo,4.0
3,0,0,Il tempo del vino e delle rose,"['Italian', 'Cafe', 'International', 'Mediterr...",333,+39 380 713 4511,http://www.facebook.com/iltempodelvinoedellerose/,256.0,4.5,"[20, 40]",...,80135.0,26,9,1,2,3,dante,VIA,Secondigliano,7.0
4,1,1,Pizza & Sfizi,"['Italian', 'Pizza', 'FastFood', 'Mediterranea...",2,+39 081 554 3128,http://www.pizzaesfizipollon.it/,348.0,5.0,"[8, 13]",...,80142.0,12,1,0,0,0,vespucci amerigo,VIA,Mercato,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2909,1695,1902,Burger King,"['Takeout', 'Seating']",2730,+39 081 580 0044,http://www.burgerking.it/,126.0,2.0,[Na],...,80132.0,0,0,1,0,6,battisti cesare,VIA,S.Giuseppe,2.0
2910,1697,1905,Bar Capodimonte,"['Seating', 'TableService']",2733,+39 081 741 0829,http://www.pizzeriacapodimonte.napoli.it/,15.0,1.5,[Na],...,80145.0,1,1,0,3,10,"(None, check)",NaN,NaN,NaN
2911,1698,1906,L.u.i.s.e.,"['Italian', 'Bar', 'Cafe', 'Mediterranean', 'N...",2738,+39 081 415367,https://www.facebook.com/gastronomialuise/,622.0,2.0,"[3, 15]",...,80134.0,4,8,8,9,22,toledo,VIA,S.Ferdinando,1.0
2912,1698,1906,L.u.i.s.e.,"['Italian', 'Bar', 'Cafe', 'Mediterranean', 'N...",2738,+39 081 415367,https://www.facebook.com/gastronomialuise/,622.0,2.0,"[3, 15]",...,80134.0,4,8,8,9,22,toledo,VIA,Montecalvario,2.0


In [20]:
dfCopy2 = dfCopy2.drop_duplicates(['CompanyName', 'Address', 'cap'])
dfCopy2 = dfCopy2.drop(columns=['level_0', 'index'])
dfCopy2 = dfCopy2.reset_index()

In [21]:
dfCopy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        1489 non-null   int64  
 1   CompanyName  1489 non-null   object 
 2   Cusine       1489 non-null   object 
 3   Importance   1489 non-null   int64  
 4   PhoneNumber  1489 non-null   object 
 5   Website      1489 non-null   object 
 6   NumReviews   1489 non-null   float64
 7   Overall      1489 non-null   float64
 8   PriceRange   1489 non-null   object 
 9   Prices       1489 non-null   object 
 10  Puntuation   1489 non-null   object 
 11  cap          1489 non-null   float64
 12  Excellent    1489 non-null   object 
 13  VeryGood     1489 non-null   object 
 14  Average      1489 non-null   object 
 15  Poor         1489 non-null   object 
 16  Terrible     1489 non-null   object 
 17  Address      1489 non-null   object 
 18  TIPOLOGIA    1369 non-null   object 
 19  QUARTI

As we could see, there were some missing data. The following loop fixes this problem. 

In [22]:
d = {'Cap':[], 'Mun':[], 'Quar':[]}
j = 0
for i in dfCopy2.cap:
    if isinstance(dfCopy2.MUN.iloc[j], float):
        if i in d['Cap']:
            n = d['Cap'].index(i)
            j = j + 1
        else:
            d['Cap'].append(i)
            d['Mun'].append(dfCopy2.MUN.iloc[j])
            d['Quar'].append(dfCopy2.QUARTIERE.iloc[j])
            j = j + 1

    else:
        j = j + 1

d = {'Cap':[], 'Mun':[], 'Quar':[]}

j = 0 
for i in dfCopy2.cap:
    if i not in d['Cap']:
        d['Cap'].append(i)
        d['Mun'].append(dfCopy2.MUN.iloc[j])
        d['Quar'].append(dfCopy2.QUARTIERE.iloc[j])
        j = j + 1
    else:
        j = j + 1

In [23]:
dfCopy2 = dfCopy2.drop(columns=['index'])

In [24]:
df_final = pd.merge(df, dfCopy2, on=['CompanyName'], how='inner')

In [25]:
df_final.columns

Index(['level_0', 'index', 'CompanyName', 'Cusine_x', 'Importance_x',
       'Address_x', 'PhoneNumber_x', 'Website_x', 'NumReviews_x', 'Overall_x',
       'PriceRange_x', 'Prices_x', 'Puntuation_x', 'cap_x', 'Excellent_x',
       'VeryGood_x', 'Average_x', 'Poor_x', 'Terrible_x', 'Address1',
       'Cusine_y', 'Importance_y', 'PhoneNumber_y', 'Website_y',
       'NumReviews_y', 'Overall_y', 'PriceRange_y', 'Prices_y', 'Puntuation_y',
       'cap_y', 'Excellent_y', 'VeryGood_y', 'Average_y', 'Poor_y',
       'Terrible_y', 'Address_y', 'TIPOLOGIA', 'QUARTIERE', 'MUN'],
      dtype='object')

In [26]:
df_final = df_final.drop(columns= ['level_0', 'index', 'Address_x', 'Address_y', 'cap_y'])
df_final = df_final.drop_duplicates(['CompanyName', 'Address1'])
df_final = df_final.rename(columns={"QUARTIERE":"Quar", "MUN":'Mun', "cap_x":"Cap"})

In [27]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1489 entries, 0 to 1533
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CompanyName    1489 non-null   object 
 1   Cusine_x       1489 non-null   object 
 2   Importance_x   1489 non-null   int64  
 3   PhoneNumber_x  1489 non-null   object 
 4   Website_x      1489 non-null   object 
 5   NumReviews_x   1489 non-null   float64
 6   Overall_x      1489 non-null   float64
 7   PriceRange_x   1489 non-null   object 
 8   Prices_x       1489 non-null   object 
 9   Puntuation_x   1489 non-null   object 
 10  Cap            1489 non-null   float64
 11  Excellent_x    1489 non-null   object 
 12  VeryGood_x     1489 non-null   object 
 13  Average_x      1489 non-null   object 
 14  Poor_x         1489 non-null   object 
 15  Terrible_x     1489 non-null   object 
 16  Address1       1489 non-null   object 
 17  Cusine_y       1489 non-null   object 
 18  Importan

In [28]:
df_final = df_final.sort_values(by='Cap').fillna(method='ffill')

Now we will create from the Cusine column a Dummy Variables dataframe.  

In [29]:
dummyCusine = pd.get_dummies(df_final['Cusine_x'].explode()).groupby(level=0).sum()

In [30]:
dummyCusine.columns = dummyCusine.columns.str.replace("'", "")

In [31]:
for i in dummyCusine.columns:
    if ((dummyCusine[i].value_counts()[1])/len(dummyCusine)) < 0.01:
        dummyCusine = dummyCusine.drop([i], axis=1)

In [32]:
nameCol = df_final.CompanyName
munCol = df_final.Mun
dummyCusine1 = pd.concat([nameCol, dummyCusine, munCol], axis=1)
dummyCusine1.columns

Index(['CompanyName', 'American', 'Asian', 'Bar', 'Barbecue', 'Breakfast',
       'BrewPub', 'Brunch', 'Cafe', 'Campania', 'Deli', 'Dinner', 'Drinks',
       'European', 'FastFood', 'Fusion', 'Gastropub', 'GlutenFreeOptions',
       'Grill', 'Healthy', 'International', 'Italian', 'Japanese', 'LateNight',
       'Lunch', 'Mediterranean', 'Neapolitan', 'Pizza', 'Pub', 'Reservations',
       'Seafood', 'Seating', 'ServesAlcohol', 'Southern-Italian', 'Steakhouse',
       'StreetFood', 'Sushi', 'TableService', 'Takeout', 'VeganOptions',
       'VegetarianFriendly', 'WheelchairAccessible', 'WineBar', 'Mun'],
      dtype='object')

In [34]:
df_final.head()

,CompanyName,Cusine_x,Importance_x,PhoneNumber_x,Website_x,NumReviews_x,Overall_x,PriceRange_x,Prices_x,Puntuation_x,...,Prices_y,Puntuation_y,Excellent_y,VeryGood_y,Average_y,Poor_y,Terrible_y,TIPOLOGIA,Quar,Mun
482,Cariño Nikkei,"['VegetarianFriendly', 'VeganOptions']",527,+39 081 1925 3881,https://www.facebook.com/carinonikkei/,139.0,4.5,[Na],[Na],"['5', '0', '0', '0', '0']",...,[Na],"['5', '0', '0', '0', '0']",5,0,0,0,0,PIAZZA,Chiaia,1.0
174,Urubamba Nikkei - Fusion - Bar,"['Peruvian', 'Seafood']",182,+39 349 813 9574,http://www.urubamba.it/,259.0,4.5,"[40, 80]","[16.0, 30.0, 16.0]","['7', '1', '1', '0', '0']",...,"[16.0, 30.0, 16.0]","['7', '1', '1', '0', '0']",7,1,1,0,0,GRADONI,Chiaia,1.0
1439,Fiorenzano 1897,"['Southern-Italian', 'Italian', 'Pizza', 'Seaf...",2383,+39 340 608 4262,http://www.fiorenzano.it/,36.0,3.0,"[15, 30]",[Na],"['0', '0', '0', '1', '0']",...,[Na],"['0', '0', '0', '1', '0']",0,0,0,1,0,VIA,S.Ferdinando,1.0
179,Antica Latteria Cucina Tipica Napoletana,"['VegetarianFriendly', 'GlutenFreeOptions']",184,+39 081 012 8775,http://www.anticalatteria.it/,765.0,4.0,[Na],[Na],"['41', '6', '3', '1', '4']",...,[Na],"['41', '6', '3', '1', '4']",41,6,3,1,4,VIA,Chiaia,1.0
408,Gusto&Gusto,"['VegetarianFriendly', 'VeganOptions', 'Gluten...",449,+39 081 245 2662,http://www.antonioeantonio.com/ristoranti/gust...,594.0,4.0,[Na],[Na],"['52', '42', '14', '8', '2']",...,[Na],"['52', '42', '14', '8', '2']",52,42,14,8,2,VIA,S.Ferdinando,1.0


In [35]:
dummyCusine1.head()

,CompanyName,American,Asian,Bar,Barbecue,Breakfast,BrewPub,Brunch,Cafe,Campania,...,Steakhouse,StreetFood,Sushi,TableService,Takeout,VeganOptions,VegetarianFriendly,WheelchairAccessible,WineBar,Mun
482,Cariño Nikkei,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1.0
174,Urubamba Nikkei - Fusion - Bar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1439,Fiorenzano 1897,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
179,Antica Latteria Cucina Tipica Napoletana,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1.0
408,Gusto&Gusto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1.0


In [36]:
df_final.to_csv('df')
dummyCusine1.to_csv('dummyCusine')